In [1]:
# 多输入多输出通道
''' 
为了加深理解，我们实现⼀下多输⼊通道互相关运算。简⽽⾔之，我们所做的就是对每个通道执⾏互相关操
作，然后将结果相加。
'''
import torch
from d2l import torch as d2l

In [2]:
'''
zip () 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，
然后返回由这些元组组成的列表。
'''
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在⼀起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K)) # 对应通道卷积后求和

In [3]:
# 我们可以构造与 图6.4.1中的值相对应的输⼊张量X和核张量K，以验证互相关运算的输出
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]]) # 两个通道
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])  # 两个2x2卷积
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
# 多输出通道
# 实现⼀个计算多个通道的输出的互相关函数
''' stack（tensors,dim=0,out=None）函数的运行机制可以等价为：
dim=0时，将tensor在一维上连接，简单来说就是，就是将tensor1，tensor2…tensor n,连接为【tensor1，tensor2… tensor n】（就是在这里产生了扩维）
dim=1时，将每个tensor的第i行按行连接组成一个新的2维tensor，再将这些新tensor按照dim=0的方式连接。
dim=2时，将每个tensor的第i行转置后按列连接组成一个新的2维tensor，再将这些新tesnor按照dim=0的方式连接
'''
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输⼊“X”执⾏互相关运算。
    # 最后将所有结果都叠加在⼀起|
    # stack() 沿着一个新维度对输入张量序列进行连接。 序列中所有的张量都应该为相同形状
    # outputs = torch.stack(inputs, dim=0) → Tensor  
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0) # stack 堆叠
''' 
for k in K: # 分别对应K,K+1（K中每个元素加1）和K+2
    print(k)
'''

' \nfor k in K: # 分别对应K,K+1（K中每个元素加1）和K+2\n    print(k)\n'

In [5]:
# 通过将核张量K与K+1（K中每个元素加1）和K+2连接起来，构造了⼀个具有3个输出通道的卷积核。
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [6]:
''' 
下⾯，我们对输⼊张量X与卷积核张量K执⾏互相关运算。现在的输出包含3个通道，第⼀个通道的结果与先
前输⼊张量X和多输⼊单输出通道的结果⼀致。
'''
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [8]:
# 1 × 1 卷积层
# 使⽤全连接层实现1 × 1卷积。请注意，我们需要对输⼊和输出的数据形状进⾏调整
''' 
图6.4.2展⽰了使⽤1 × 1卷积核与3个输⼊通道和2个输出通道的互相关计算。这⾥输⼊和输出具有相同的⾼度
和宽度，输出中的每个元素都是从输⼊图像中同⼀位置的元素的线性组合。我们可以将1 × 1卷积层看作是在
每个像素位置应⽤的全连接层，以ci个输⼊值转换为co个输出值。因为这仍然是⼀个卷积层，所以跨像素的
权重是⼀致的。同时，1 × 1卷积层需要的权重维度为co × ci，再额外加上⼀个偏置。
'''
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape 
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [9]:
''' 
当执⾏1 × 1卷积运算时，上述函数相当于先前实现的互相关函数corr2d_multi_in_out。让我们⽤⼀些
样本数据来验证这⼀点
'''
X = torch.normal(0, 1, (3, 3, 3)) # torch.normal(means, std, out=None)
K = torch.normal(0, 1, (2, 3, 1, 1)) # 均值0,方差1 (2, 3, 1, 1) 输入2通道,输出3,大小1x1

In [10]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [ ]:
''' 
 ⼩结
• 多输⼊多输出通道可以⽤来扩展卷积层的模型。
• 当以每像素为基础应⽤时，1 × 1卷积层相当于全连接层。
• 1 × 1卷积层通常⽤于调整⽹络层的通道数量和控制模型复杂性。
'''